In [8]:
import csv
import pandas as pd
import unicodedata
import os


In [10]:
def hebStripped(word):

    normalized = unicodedata.normalize('NFKD', word)

    return ''.join([c for c in normalized if not unicodedata.combining(c)])

In [241]:
def getAlignedIndeces(tableA, rowA, colA, tableB, rowB, colB):

    _rowA = rowA
    _rowB = rowB

    dist = 3
    n = 0 

    while n < dist:

        rowA += n 
        rowB += n

        tries = 3
        i = 1

        while i < tries:
            
            wordA = hebStripped(tableA[rowA][colA])
            wordB = hebStripped(tableB[rowB+i][colB])
            print('A', [wordA, wordB])
            if wordA in wordB and wordA != '':
                print('A', wordA, rowA, rowB+i)
                
                return rowA, rowB+i

            i += 1

        i = 0
        while i < tries:
            
            wordA = hebStripped(tableA[rowA+i][colA])
            wordB = hebStripped(tableB[rowB][colB])
            print('B', [wordA, wordB])
            if wordB in wordA and wordB != '':
                print(wordB, rowA+i, rowB)

                return rowA+i, rowB

            i += 1

        n += 1

    return 0

In [251]:
# TODO

'''
EXO 21:8!7,ל֥,לֹא,False
EXO 21:8!7,NA,יְעָדָ֖,True
EXO 21:8!7,NA,הּ,True
EXO 21:8!7,וֹ,וְ
EXO 21:8!8,יְעָדָ֖,הֶפְדָּ֑,False
EXO 21:8!8,הּ,הּ
EXO 21:8!9,ו,ל,?
EXO 21:8!9,הֶפְדָּ֑,עַ֥ם
EXO 21:8!9,הּ,נָכְרִ֛י,False
EXO 21:8!10,NA,לֹא,True
EXO 21:8!10,לְ,יִמְשֹׁ֥ל
EXO 21:8!10,עַ֥ם,NA,False
EXO 21:8!11,נָכְרִ֛י,NA,False
EXO 21:8!12,לֹא,לְ
EXO 21:8!13,יִמְשֹׁ֥ל,NA,False
EXO 21:8!14,לְ,NA,False
'''

# TODO : check if the next val is equal.
# a[i+1] == b[i+1]
'''
EXO 28:28!5,טבעתי,טבעת,True
EXO 28:28!5,NA,וֹ,True

NUM 23:13!5,לכ,לך,True
NUM 23:13!5,ה,נָּ֨א
NUM 23:13!6,נָּ֨א,NA,False
'''

def compareStepToMacula(mTable, mwCol, sTable, swCol):

    rows = []

    mismatches = {}

    mRow = 1
    sRow = 1

    length = min(len(mTable), len(sTable))
    
    while sRow < length:

        ref = mTable[mRow][1]
        mw = mTable[mRow][mwCol]
        sw = sTable[sRow][swCol]
        
        mw = hebStripped(mw)
        sw = hebStripped(sw)

        if ref == 'RUT 3:17!9' and mw == 'אל':
            rows.append([ref, mw, 'NA', 'True'])
            rows.append([ref, 'י', 'NA', 'True'])
            mRow += 2
            sRow += 0
            continue

        if mw != sw:

            nextIsEqual = hebStripped(mTable[mRow+1][mwCol]) == hebStripped(sTable[sRow+1][swCol])

            if nextIsEqual:
                rows.append([ref, mw, sw, True])
                mRow += 1
                sRow += 1
                continue


            print(ref, mRow, sRow, [mw, sw])

            # Likely a difference in suffix marking, like Gen 9:21!7
            # if len(mw) == 1 and len(sw) == 1:

            #     rows.append([ref, mw, sw, '?'])
            #     mRow += 1
            #     sRow += 1
            
            # Likely absence of a suffix, like Gen 15:2!3
            # if len(mw) > 2 and len(sw) > 2 and mw[:2] == sw[:2]:
            if len(mw) > 0 and len(sw) > 0 and mw[0] == sw[0]:
                 
                rows.append([ref, mw, sw, True])

                # Likely dif in ketiv, e.g., Gen 14:2!17 ['צבויים' ,'צבוים']
                if mw[-1] == sw[-1] or len(mw) == len(sw):
                    mRow += 1
                    sRow += 1
                
                elif len(mw) > len(sw):
                    
                    rows.append([
                        ref,
                        'NA',
                        sTable[sRow+1][swCol],
                        True
                    ])

                    mRow += 1
                    sRow += 2
                
                else:

                    rows.append([
                        ref,
                        mTable[mRow+1][mwCol],
                        'NA',
                        True
                    ])

                    mRow += 2
                    sRow += 1

            else:

                newRowIndeces = getAlignedIndeces(mTable, mRow, mwCol, sTable, sRow, swCol)

                print(mRow, sRow, newRowIndeces)
                if not newRowIndeces:
                    rows.append([ref, mw, sw, "FAILED"])
                    return rows
                
                newMRow, newSRow = newRowIndeces

                if newMRow - mRow > newSRow - sRow:

                    while sRow < newSRow:

                        ref = mTable[mRow][1]
                        mw = mTable[mRow][mwCol]
                        sw = sTable[sRow][swCol]

                        rows.append([ref, mw, sw, False])

                        mRow += 1
                        sRow += 1

                    while mRow < newMRow:

                        ref = mTable[mRow][1]
                        mw = mTable[mRow][mwCol]
                        sw = sTable[sRow][swCol]

                        rows.append([ref, mw, 'NA', False])

                        mRow += 1
                
                else:

                    while mRow < newMRow:

                        ref = mTable[mRow][1]
                        mw = mTable[mRow][mwCol]
                        sw = sTable[sRow][swCol]

                        rows.append([ref, mw, sw, False])

                        mRow += 1
                        sRow += 1

                    while sRow < newSRow:

                        ref = mTable[mRow][1]
                        mw = mTable[mRow][mwCol]
                        sw = sTable[sRow][swCol]

                        rows.append([ref, 'NA', sw, True])

                        sRow += 1
            
        rows.append([
            mTable[mRow][1],
            mTable[mRow][mwCol],
            sTable[sRow][swCol],
        ])

        mRow += 1
        sRow += 1

    return rows


In [ ]:
rows = compareStepToMacula(macula, 5, step, 3)

header = ['ref', 'macula', 'step']

with open('comp4.csv', 'w', encoding='UTF8', newline='') as f:
    
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(rows)